# EEE102 Activity 2

This activity is an application of Newton-Raphson for univariate root-finding.

## Imports

In [1]:
import math as mt

import scipy.optimize as spo

import numpy as np

## Scenario

You are a power systems engineer tasked to analyze two customer loads and served at a common bus.
The bus is at the receiving end of a feeder line extending from the utility system.

One of the loads is estimated to be 400 kilowatts at a lagging power factor of 0.85,
while the other is rated 300 kilovars at a lagging power factor of 0.75.

Records show that the feeder line impedance is rated 1.00 + j2.50 ohms.
If it is reasonable to assume that the utility system's RMS voltage hovers around 13.8 kilovolts,
at what RMS voltage are the loads served?

You are to approach this as a root-finding problem,
and so must first derive a function $f\!\left(V\right)$
whose root corresponds to the desired quantity.

## Newton-Raphson

Construct two Python functions:
- `f()`, which implements $f\!\left(V\right)$, and
- `df_dV()`, which implements $f^{\prime}\!\left(V\right)$
  (*i.e.*, the derivative of $f\!\left(V\right)$ w.r.t. $V$).

Each of the said Python functions takes a single positional argument `x`,
representing $V$.

In [2]:
def f(V):
    V_ph = complex(V, 0)

    P1 = 400e3
    pf1 = 0.85
    phi1 = np.arccos(pf1)
    Q1 = P1 * np.tan(phi1)

    Q2 = 300e3
    pf2 = 0.75
    phi2 = np.arccos(pf2)
    P2 = Q2 / np.tan(phi2)

    S1 = P1 + 1j * Q1
    S2 = P2 + 1j * Q2
    S_total = S1 + S2

    Z = 1.0 + 2.5j
    Vs = 13.8e3

    I = np.conjugate(S_total / V_ph)
    Vs_calc = V_ph + I * Z

    return abs(Vs_calc) - Vs

def df_dV(V):
    h = 1e-6
    return (f(V + h) - f(V - h)) / (2 * h)

Use Newton-Raphson to find a root of $f\!\left(V\right)$.
Use an iteration budget of 42 and a tolerance of $10^{-7}$.

Define the following variables.
- `X0`, to store the initial guess
- `MAX_ITERS`, to store the iteration budget
- `X_TOL` to store the tolerance

Run
[`scipy.optimize.newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html)
such that you get the (approximate) root as well as information concerning the run.
Store the root and the information in variables `p_nr` and `p_nr_info`, respectively.
Make sure to verify if the returned root is indeed a root.

In [3]:
# Initial guess - a reasonable starting point would be slightly below the utility voltage
X0 = 13000  # Starting with 13 kV as initial guess
MAX_ITERS = 42  # Maximum iterations as specified
X_TOL = 1e-7  # Tolerance as specified

# Using scipy's newton method to find the root
p_nr, p_nr_info = spo.newton(f, X0, fprime=df_dV, maxiter=MAX_ITERS, 
                              tol=X_TOL, full_output=True)

# Verify if the returned value is indeed a root
root_verification = f(p_nr)
print(f"Newton-Raphson solution: {p_nr:.4f} V")
print(f"Function value at solution: {root_verification:.8e}")
print(f"Iterations required: {p_nr_info.iterations}")
print(f"Function evaluations: {p_nr_info.function_calls}")

# Convert to kV for easier interpretation
print(f"Load bus voltage: {p_nr/1000:.4f} kV")

# Additional verification - calculate the voltage drop across the line
V_ph = complex(p_nr, 0)

P1 = 400e3
pf1 = 0.85
phi1 = np.arccos(pf1)
Q1 = P1 * np.tan(phi1)

Q2 = 300e3
pf2 = 0.75
phi2 = np.arccos(pf2)
P2 = Q2 / np.tan(phi2)

S_total = (P1 + 1j * Q1) + (P2 + 1j * Q2)
I = np.conjugate(S_total / V_ph)
Z = 1.0 + 2.5j
V_drop = I * Z
Vs_calc = V_ph + V_drop

print(f"Calculated sending-end voltage: {abs(Vs_calc)/1000:.4f} kV")
print(f"Expected sending-end voltage: 13.8000 kV")
print(f"Voltage drop across line: {abs(V_drop)/1000:.4f} kV")

# Display load data
print(f"\nLoad 1: {P1/1000:.1f} kW + j{Q1/1000:.1f} kvar")
print(f"Load 2: {P2/1000:.1f} kW + j{Q2/1000:.1f} kvar")
print(f"Total load: {S_total.real/1000:.1f} kW + j{S_total.imag/1000:.1f} kvar")

Newton-Raphson solution: 13645.0413 V
Function value at solution: -3.63797881e-12
Iterations required: 3
Function evaluations: 6
Load bus voltage: 13.6450 kV
Calculated sending-end voltage: 13.8000 kV
Expected sending-end voltage: 13.8000 kV
Voltage drop across line: 0.1817 kV

Load 1: 400.0 kW + j247.9 kvar
Load 2: 340.2 kW + j300.0 kvar
Total load: 740.2 kW + j547.9 kvar


## Results

Do not modify nor remove the following code cell.

In [4]:
print(f"Finding a root from {X0}:")
print(p_nr_info)
print(f"Residual value: {f(p_nr)}")

Finding a root from 13000:
      converged: True
           flag: converged
 function_calls: 6
     iterations: 3
           root: 13645.041339303667
         method: newton
Residual value: -3.637978807091713e-12


In [5]:
print(f"The RMS voltage at the common bus is {p_nr} kilovolts.")

The RMS voltage at the common bus is 13645.041339303667 kilovolts.


## Submission instructions

Download this notebook file,
and save with a filename following the pattern
`ACT-02_<section>_<ID number>`,
where the section is as reflected in your Google Classroom.
For example, if your ID number is 2013-0024
and you are enrolled to the M34W12 class,
then your notebook should be named `ACT-02_M34W12_2013-0024.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-10-01*